This notebook shows the MEP quickstart sample, which also exists as a non-notebook version at:
https://bitbucket.org/vitotap/python-spark-quickstart

It shows how to use Spark (http://spark.apache.org/) for distributed processing on the PROBA-V Mission Exploitation Platform. (https://proba-v-mep.esa.int/) The sample intentionally implements a very simple computation: for each PROBA-V tile in a given bounding box and time range, a histogram is computed. The results are then summed and printed. Computation of the histograms runs in parallel.

In [1]:
# List of files for which a histogram needs to be calculated. Each file should be a single-band file
# supported by GDAL.
base = "/data/MTDA/TIFFDERIVED/PROBAV_L3_S1_TOC_333M/2016/20160101/PROBAV_S1_TOC_20160101_333M_V101/"
files = [
    base + "PROBAV_S1_TOC_X18Y02_20160101_333M_V101_NDVI.tif"
]
#check if file exists
!file /data/MTDA/TIFFDERIVED/PROBAV_L3_S1_TOC_333M/2016/20160101/PROBAV_S1_TOC_20160101_333M_V101/PROBAV_S1_TOC_X18Y02_20160101_333M_V101_NDVI.tif


/data/MTDA/TIFFDERIVED/PROBAV_L3_S1_TOC_333M/2016/20160101/PROBAV_S1_TOC_20160101_333M_V101/PROBAV_S1_TOC_X18Y02_20160101_333M_V101_NDVI.tif: TIFF image data, little-endian


In [2]:
# Calculates the histogram for a given (single band) image file.
def histogram(image_file):
    
    import numpy as np
    import gdal
    
    
    # Open image file
    img = gdal.Open(image_file)
    
    if img is None:
        print '-ERROR- Unable to open image file "%s"' % image_file
    
    # Open raster band (first band)
    raster = img.GetRasterBand(1)    
    xSize = img.RasterXSize
    ySize = img.RasterYSize
    
    # Read raster data
    data = raster.ReadAsArray(0, 0, xSize, ySize)
        
    # Calculate histogram
    hist, _ = np.histogram(data, bins=256)
    return hist


In [3]:
# ================================================================
# === Calculate the histogram for a given number of files. The ===
# === processing is performed by spreading them over a cluster ===
# === of Spark nodes.                                          ===
# ================================================================

from datetime import datetime
from operator import add
import pyspark

# Setup the Spark cluster
conf = pyspark.SparkConf()
conf.set('spark.yarn.executor.memoryOverhead', 1024)
conf.set('spark.executor.memory', '8g')
conf.set('spark.executor.cores', '2')
conf.set('spark.executor.instances', 10)
sc = pyspark.SparkContext(conf=conf)

In [4]:
# Distribute the local file list over the cluster.
filesRDD = sc.parallelize(files)

# Apply the 'histogram' function to each filename using 'map', keep the result in memory using 'cache'.
hists = filesRDD.map(histogram).cache()

count = hists.count()

# Combine distributed histograms into a single result
total = hists.reduce(lambda h, i: map(add, h, i))

print "Sum of %i histograms: %s" % (count, total)

Sum of 1 histograms: [408984  30269  32025  34376  36155  37423  38609  41546  42960  45630
  47651  50622  51609  53812  56539  61465  67533  78156  91818 115013
 150011 195501 272860 443835 477491 374683 354246 355732 299753 255058
 216026 194968 180004 166236 159434 156889 153027 145268 135398 126976
 120927 114560 108331 102574  98048  93372  89243  85580  81764  77958
  74907  71233  68395  65158  62921  59645  57352  55044  52580  50710
  48622  47208  45937  45037  43640  42467  41242  40517  39134  39044
  37808  37000  36221  35918  34833  34014  33538  33134  32151  32184
  31528  31282  30121  30743  30038  29353  29312  28890  28222  28180
  28314  28091  27415  27719  27565  27138  27257  27042  27097  26892
  26847  27260  27145  27034  26889  27431  27224  27326  27140  27566
  27578  27578  27806  27824  27676  28040  28206  28308  27982  28565
  28070  28042  28323  28304  28359  28178  28179  27956  27896  28012
  28079  27695  27727  27328  27326  26929  26992  26813